## Embeddings

No nosso exemplo anterior, trabalhámos com vetores de bag-of-words de alta dimensão com comprimento `vocab_size`, e convertemos explicitamente vetores de representação posicional de baixa dimensão em representações esparsas de uma só posição ativa (one-hot). Esta representação one-hot não é eficiente em termos de memória. Além disso, cada palavra é tratada de forma independente, o que significa que os vetores codificados em one-hot não expressam semelhanças semânticas entre palavras.

Nesta unidade, continuaremos a explorar o conjunto de dados **News AG**. Para começar, vamos carregar os dados e obter algumas definições da unidade anterior.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### O que é uma embedding?

A ideia de **embedding** é representar palavras utilizando vetores densos de dimensão reduzida que refletem o significado semântico da palavra. Mais à frente, discutiremos como construir embeddings de palavras significativas, mas por agora vamos apenas pensar em embeddings como uma forma de reduzir a dimensionalidade de um vetor de palavras.

Assim, uma camada de embedding recebe uma palavra como entrada e produz um vetor de saída com o tamanho especificado por `embedding_size`. De certa forma, é muito semelhante a uma camada `Dense`, mas em vez de receber um vetor codificado em one-hot como entrada, consegue receber um número que representa a palavra.

Ao usar uma camada de embedding como a primeira camada na nossa rede, podemos mudar de um modelo de bag-of-words para um modelo de **embedding bag**, onde primeiro convertemos cada palavra do nosso texto na embedding correspondente e, em seguida, calculamos uma função agregada sobre todas essas embeddings, como `sum`, `average` ou `max`.

![Imagem mostrando um classificador com embedding para cinco palavras de sequência.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

A nossa rede neural de classificação consiste nas seguintes camadas:

* Camada `TextVectorization`, que recebe uma string como entrada e produz um tensor de números de tokens. Vamos especificar um tamanho de vocabulário razoável, `vocab_size`, e ignorar palavras menos usadas. A forma da entrada será 1, e a forma da saída será $n$, já que obteremos $n$ tokens como resultado, cada um contendo números entre 0 e `vocab_size`.
* Camada `Embedding`, que recebe $n$ números e reduz cada número a um vetor denso de um comprimento especificado (100 no nosso exemplo). Assim, o tensor de entrada com forma $n$ será transformado num tensor $n\times 100$.
* Camada de agregação, que calcula a média deste tensor ao longo do primeiro eixo, ou seja, calculará a média de todos os $n$ tensores de entrada correspondentes a diferentes palavras. Para implementar esta camada, usaremos uma camada `Lambda` e passaremos para ela a função para calcular a média. A saída terá uma forma de 100 e será a representação numérica de toda a sequência de entrada.
* Classificador linear final `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


No resumo `summary`, na coluna **output shape**, a primeira dimensão do tensor `None` corresponde ao tamanho do minibatch, e a segunda corresponde ao comprimento da sequência de tokens. Todas as sequências de tokens no minibatch têm comprimentos diferentes. Vamos discutir como lidar com isso na próxima secção.

Agora, vamos treinar a rede:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Nota** que estamos a construir o vetorizador com base num subconjunto dos dados. Isto é feito para acelerar o processo, e pode resultar numa situação em que nem todos os tokens do nosso texto estejam presentes no vocabulário. Neste caso, esses tokens seriam ignorados, o que pode resultar numa precisão ligeiramente inferior. No entanto, na vida real, um subconjunto de texto frequentemente fornece uma boa estimativa do vocabulário.


### Lidando com tamanhos variáveis de sequências

Vamos entender como o treino ocorre em minibatches. No exemplo acima, o tensor de entrada tem dimensão 1, e usamos minibatches de tamanho 128, de forma que o tamanho real do tensor é $128 \times 1$. No entanto, o número de tokens em cada frase é diferente. Se aplicarmos a camada `TextVectorization` a uma única entrada, o número de tokens retornados será diferente, dependendo de como o texto é tokenizado:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


No entanto, quando aplicamos o vetorizador a várias sequências, ele tem de produzir um tensor de forma retangular, preenchendo os elementos não utilizados com o token PAD (que, no nosso caso, é zero):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Aqui podemos ver as incorporações:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Nota**: Para minimizar a quantidade de preenchimento, em alguns casos faz sentido ordenar todas as sequências no conjunto de dados pela ordem crescente de comprimento (ou, mais precisamente, pelo número de tokens). Isto garantirá que cada minibatch contenha sequências de comprimento semelhante.


## Embeddings semânticos: Word2Vec

No nosso exemplo anterior, a camada de embedding aprendeu a mapear palavras para representações vetoriais; no entanto, essas representações não tinham significado semântico. Seria interessante aprender uma representação vetorial em que palavras semelhantes ou sinónimos correspondam a vetores próximos entre si em termos de alguma distância vetorial (por exemplo, distância euclidiana).

Para isso, precisamos de pré-treinar o nosso modelo de embedding numa grande coleção de texto utilizando uma técnica como o [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Esta técnica baseia-se em duas arquiteturas principais que são usadas para produzir uma representação distribuída de palavras:

 - **Continuous bag-of-words** (CBoW), onde treinamos o modelo para prever uma palavra com base no contexto envolvente. Dado o ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, o objetivo do modelo é prever $W_0$ a partir de $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** é o oposto do CBoW. O modelo utiliza a janela de palavras de contexto envolvente para prever a palavra atual.

O CBoW é mais rápido, enquanto o skip-gram, embora mais lento, representa melhor palavras menos frequentes.

![Imagem mostrando os algoritmos CBoW e Skip-Gram para converter palavras em vetores.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Para experimentar o embedding Word2Vec pré-treinado no conjunto de dados Google News, podemos usar a biblioteca **gensim**. Abaixo, encontramos as palavras mais semelhantes a 'neural'.

> **Nota:** Quando cria vetores de palavras pela primeira vez, o download pode demorar algum tempo!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Podemos também extrair a incorporação vetorial da palavra, para ser utilizada no treino do modelo de classificação. A incorporação tem 300 componentes, mas aqui mostramos apenas os primeiros 20 componentes do vetor para maior clareza:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

A grande vantagem das incorporações semânticas é que se pode manipular a codificação vetorial com base na semântica. Por exemplo, podemos pedir para encontrar uma palavra cuja representação vetorial esteja o mais próxima possível das palavras *rei* e *mulher*, e o mais distante possível da palavra *homem*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Um exemplo acima utiliza alguma magia interna do GenSym, mas a lógica subjacente é na verdade bastante simples. Uma coisa interessante sobre embeddings é que se podem realizar operações normais de vetores em vetores de embedding, e isso refletiria operações nos **significados** das palavras. O exemplo acima pode ser expresso em termos de operações de vetores: calculamos o vetor correspondente a **REI-HOMEM+MULHER** (as operações `+` e `-` são realizadas nas representações vetoriais das palavras correspondentes), e depois encontramos a palavra mais próxima no dicionário para esse vetor:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTA**: Tivemos de adicionar pequenos coeficientes aos vetores *man* e *woman* - experimente removê-los para ver o que acontece.

Para encontrar o vetor mais próximo, utilizamos a estrutura do TensorFlow para calcular um vetor de distâncias entre o nosso vetor e todos os vetores no vocabulário, e depois encontramos o índice da palavra mínima usando `argmin`.


Embora o Word2Vec pareça uma ótima forma de expressar a semântica das palavras, apresenta várias desvantagens, incluindo as seguintes:

* Tanto os modelos CBoW como os skip-gram são **embeddings preditivos**, e consideram apenas o contexto local. O Word2Vec não tira proveito do contexto global.
* O Word2Vec não leva em conta a **morfologia** das palavras, ou seja, o facto de o significado de uma palavra poder depender de diferentes partes da mesma, como a raiz.

O **FastText** tenta superar a segunda limitação e baseia-se no Word2Vec ao aprender representações vetoriais para cada palavra e para os n-gramas de caracteres encontrados dentro de cada palavra. Os valores das representações são então calculados como uma média, resultando num único vetor em cada etapa de treino. Embora isso adicione um grande volume de computação adicional ao pré-treino, permite que os embeddings de palavras codifiquem informações de subpalavras.

Outro método, o **GloVe**, utiliza uma abordagem diferente para embeddings de palavras, baseada na fatorização da matriz de contexto de palavras. Primeiro, constrói-se uma grande matriz que conta o número de ocorrências de palavras em diferentes contextos, e depois tenta-se representar essa matriz em dimensões reduzidas de forma a minimizar a perda de reconstrução.

A biblioteca gensim suporta esses embeddings de palavras, e pode experimentar com eles alterando o código de carregamento do modelo acima.


## Utilizar embeddings pré-treinados no Keras

Podemos modificar o exemplo acima para pré-preencher a matriz na nossa camada de embedding com embeddings semânticos, como o Word2Vec. É provável que os vocabulários do embedding pré-treinado e do corpus de texto não coincidam, por isso precisamos escolher um deles. Aqui exploramos as duas opções possíveis: usar o vocabulário do tokenizer e usar o vocabulário dos embeddings do Word2Vec.

### Usar o vocabulário do tokenizer

Ao usar o vocabulário do tokenizer, algumas palavras do vocabulário terão embeddings correspondentes do Word2Vec, enquanto outras estarão em falta. Dado que o tamanho do nosso vocabulário é `vocab_size`, e o comprimento do vetor de embedding do Word2Vec é `embed_size`, a camada de embedding será representada por uma matriz de pesos com a forma `vocab_size`$\times$`embed_size`. Vamos preencher esta matriz percorrendo o vocabulário:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Para palavras que não estão presentes no vocabulário do Word2Vec, podemos deixá-las como zeros ou gerar um vetor aleatório.

Agora podemos definir uma camada de incorporação com pesos pré-treinados:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Nota**: Repare que definimos `trainable=False` ao criar o `Embedding`, o que significa que não estamos a re-treinar a camada Embedding. Isto pode fazer com que a precisão seja ligeiramente inferior, mas acelera o processo de treino.

### Utilizar o vocabulário do embedding

Um problema com a abordagem anterior é que os vocabulários usados no TextVectorization e no Embedding são diferentes. Para resolver este problema, podemos usar uma das seguintes soluções:
* Re-treinar o modelo Word2Vec com o nosso vocabulário.
* Carregar o nosso conjunto de dados com o vocabulário do modelo Word2Vec pré-treinado. Os vocabulários usados para carregar o conjunto de dados podem ser especificados durante o carregamento.

A segunda abordagem parece mais simples, por isso vamos implementá-la. Antes de mais, iremos criar uma camada `TextVectorization` com o vocabulário especificado, retirado dos embeddings do Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

A biblioteca de embeddings de palavras gensim contém uma função conveniente, `get_keras_embeddings`, que criará automaticamente a camada de embeddings correspondente do Keras para si.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Uma das razões pelas quais não estamos a obter maior precisão é porque algumas palavras do nosso conjunto de dados estão ausentes no vocabulário pré-treinado do GloVe e, assim, são essencialmente ignoradas. Para superar isto, podemos treinar os nossos próprios embeddings com base no nosso conjunto de dados.


## Embeddings contextuais

Uma das principais limitações das representações tradicionais de embeddings pré-treinados, como o Word2Vec, é o facto de que, embora consigam captar algum significado de uma palavra, não conseguem diferenciar entre significados diferentes. Isto pode causar problemas em modelos subsequentes.

Por exemplo, a palavra 'play' tem significados diferentes nestas duas frases:
- Fui a uma **peça** no teatro.
- O João quer **brincar** com os amigos.

Os embeddings pré-treinados de que falámos representam ambos os significados da palavra 'play' no mesmo embedding. Para superar esta limitação, precisamos de construir embeddings baseados no **modelo de linguagem**, que é treinado num grande corpus de texto e *sabe* como as palavras podem ser combinadas em diferentes contextos. Discutir embeddings contextuais está fora do âmbito deste tutorial, mas voltaremos a este tema ao falar sobre modelos de linguagem na próxima unidade.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
